In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

def PISO(n):
    @si.coroutine(inputs={"PI": si.Bits(n), "SI": si.Bit, "LOAD": si.Bit})
    def PISO():
        values = bits(0, n)
        O = values[-1]
        while True:
            PI, SI, LOAD = yield O
            O = values[-1]
            if LOAD:
                values = PI
            else:
                # values = [SI] + values[:-1]
                for i in range(n - 1, 0, -1):
                    values[i] = values[i - 1]
                values[0] = SI
    return PISO()
        
PISO10 = si.compile(PISO(10), file_name="build/silica_piso.py")
with open("build/silica_piso.py", "r") as magma_file:
    print(magma_file.read())

1: from magma import *
2: import os
3: os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
4: from mantle import *
5: import mantle.common.operator
6: 
7: PISO = DefineCircuit("PISO", "O", Out(Bit), "PI", In(Bits(10)), "SI", In(Bit), "LOAD", In(Bit), *ClockInterface(has_ce=False))
8: CE = VCC
9: Buffer = DefineCircuit("__silica_BufferPISO", "I", In(Bits(2)), "O", Out(Bits(2)))
10: wire(Buffer.I, Buffer.O)
11: EndDefine()
12: __silica_path_state = Buffer()
13: PI = PISO.PI
14: SI = PISO.SI
15: LOAD = PISO.LOAD
16: values = Register(10, has_ce=True)
17: values_CE = [CE]
18: wireclock(PISO, values)
19: values_next = Or(2, 10)
20: wire(values_next.O, values.I)
21: values_next_0 = And(2, 10)
22: wire(values_next_0.O, values_next.I0)
23: wire(values_next_0.I0[0], __silica_path_state.O[0])
24: wire(values_next_0.I0[1], __silica_path_state.O[0])
25: wire(values_next_0.I0[2], __silica_path_state.O[0])
26: wire(values_next_0.I0[3], __silica_path_state.O[0])
27: wire(values_next_0.I0[4], __silic

In [2]:
from loam.boards.icestick import IceStick
from mantle import Counter

icestick = IceStick()
icestick.Clock.on()

for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()

main = icestick.main()
piso = PISO10()
m.wire(piso.O, main.J1[0])
m.wire(m.bits(0, 7), main.J1[1:])
m.wire(piso.SI, 0)
m.wire(piso.PI, m.bits(0xFF, 10))
counter = Counter(8)
m.wire(piso.LOAD, counter.COUT)

m.compile("build/silica_piso", main)

compiling __silica_BufferPISO
compiling Register10CE
compiling Or2
compiling Or2x10
compiling And2
compiling And2x10
compiling PISO
compiling FullAdder
compiling Add8Cout
compiling Register8
compiling Counter8
compiling main


In [3]:
%%bash
yosys -p 'synth_ice40 -top main -blif build/silica_piso.blif' build/silica_piso.v
arachne-pnr -d 1k -o build/silica_piso.txt -p build/silica_piso.pcf build/silica_piso.blif
icepack build/silica_piso.txt build/silica_piso.bin
icetime -tmd hx1k build/silica_piso.txt


 /----------------------------------------------------------------------------\
 |                                                                            |
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |                                                                            |
 |  Copyright (C) 2012 - 2016  Clifford Wolf <clifford@clifford.at>           |
 |                                                                            |
 |  Permission to use, copy, modify, and/or distribute this software for any  |
 |  purpose with or without fee is hereby granted, provided that the above    |
 |  copyright notice and this permission notice appear in all copies.         |
 |                                                                            |
 |  THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES  |
 |  WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF          |
 |  MERCHANTABILITY AND FITNESS. IN NO 

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_piso.blif...
prune...
read_pcf build/silica_piso.pcf...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          51 / 1280
  DFF        18
  CARRY      8
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 8
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLKIN$2, 18 / 18
  promoted 1 nets
    1 clk
  1 globals
    1 clk
realize_constants...
  realized 1
place...
  initial wire length = 517
  at iteration #50: temp = 5.72537, wire length = 327
  at iteration #100: temp = 2.65252, wire length = 177
  at iteration #150: temp = 0.800749, wire length = 107
  at iteration #200: temp = 0.014425, wire length = 47
  final wire length = 47

After placement:
PIOs       5 / 96
PLBs       13 / 160
BRAMs     

In [4]:
from loam.boards.icestick import IceStick
from mantle import PISO

icestick = IceStick()
icestick.Clock.on()

for i in range(len(icestick.J1)):
    icestick.J1[i].output().on()

main = icestick.main()
piso = PISO(10)
m.wire(piso.O, main.J1[0])
m.wire(m.bits(0, 7), main.J1[1:])
m.wire(piso.SI, 0)
m.wire(piso.PI, m.bits(0xFF, 10))
counter = Counter(8)
m.wire(piso.LOAD, counter.COUT)

m.compile("build/mantle_piso", main)

compiling Mux2
compiling Register10
compiling PISO10
compiling FullAdder
compiling Add8Cout
compiling Register8
compiling Counter8
compiling main


In [5]:
%%bash
yosys -p 'synth_ice40 -top main -blif build/mantle_piso.blif' build/mantle_piso.v
arachne-pnr -d 1k -o build/mantle_piso.txt -p build/mantle_piso.pcf build/mantle_piso.blif
icepack build/mantle_piso.txt build/mantle_piso.bin
icetime -tmd hx1k build/mantle_piso.txt


 /----------------------------------------------------------------------------\
 |                                                                            |
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |                                                                            |
 |  Copyright (C) 2012 - 2016  Clifford Wolf <clifford@clifford.at>           |
 |                                                                            |
 |  Permission to use, copy, modify, and/or distribute this software for any  |
 |  purpose with or without fee is hereby granted, provided that the above    |
 |  copyright notice and this permission notice appear in all copies.         |
 |                                                                            |
 |  THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES  |
 |  WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF          |
 |  MERCHANTABILITY AND FITNESS. IN NO 

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/mantle_piso.blif...
prune...
read_pcf build/mantle_piso.pcf...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          27 / 1280
  DFF        18
  CARRY      8
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 8
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLKIN$2, 18 / 18
  promoted 1 nets
    1 clk
  1 globals
    1 clk
realize_constants...
  realized 1
place...
  initial wire length = 411
  at iteration #50: temp = 8.22135, wire length = 156
  at iteration #100: temp = 3.43752, wire length = 106
  at iteration #150: temp = 1.17069, wire length = 63
  at iteration #200: temp = 0.0282521, wire length = 28
  final wire length = 28

After placement:
PIOs       5 / 96
PLBs       8 / 160
BRAMs      0